In [10]:
import importlib
importlib.reload(chain)

<module 'chain' from 'e:\\Document\\Project\\ffa\\chain.py'>

In [ ]:
import dash
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import chain

app = Dash(external_stylesheets=[dbc.themes.FLATLY],  
           suppress_callback_exceptions=True,         
           prevent_initial_callbacks='initial_duplicate')

nav_bar = dbc.Nav(
    [
        dbc.NavItem(dbc.NavLink("市场全景", active=True, href="/"), ),
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("钢铁产业链", href="/chain/black_metals"), 
                dbc.DropdownMenuItem("贵金属产业链", href="/chain/precious_metals"),
                dbc.DropdownMenuItem("铝产业链", href="/chain/alum"),
                dbc.DropdownMenuItem("铜产业链", href="/chain/copper"),
             ],
            label="金属",
            nav=True,
        ),
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("动力煤产业链", href="/chain/coal"), 
                dbc.DropdownMenuItem("石油产业链", href="/chain/oil"),
                dbc.DropdownMenuItem("原油产业链", href="/chain/crude"),
             ],
            label="能源",
            nav=True,
        ),
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("化工产业链", href="/chain/chemical"),
                dbc.DropdownMenuItem("沥青产业链", href="/chain/asphalt"),
                dbc.DropdownMenuItem("橡胶产业链", href="/chain/rubber"),
             ],
            label="化工",
            nav=True,
        ),        
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("菜籽油产业链", href="/chain/rapeseed_oil"), 
                dbc.DropdownMenuItem("大豆产业链", href="/chain/soybeans"),
                dbc.DropdownMenuItem("糖产业链", href="/chain/sugar"),
                dbc.DropdownMenuItem("小麦产业链", href="/chain/wheat"),
                dbc.DropdownMenuItem("玉米产业链", href="/chain/corn"),
                dbc.DropdownMenuItem("棕榈油产业链", href="/chain/palm_oil"),
                dbc.DropdownMenuItem("生猪产业链", href="/chain/pig"),
             ],
            label="农产品",
            nav=True,
        ),
        dbc.NavItem(dbc.NavLink("自选", active=True, href="#"), ),
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("数据管理", href="/data_manage"),
                dbc.DropdownMenuItem("品种管理", href="/variety_manage"),
                dbc.DropdownMenuItem("设置", href="/setting"),
             ],
            label="我的",
            nav=True,
        ),                
    ],
    horizontal='end'
)

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "1rem",
    "margin-right": "1rem",
    # "padding": "2rem 1rem",
}

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = dbc.Container(
    [
        dcc.Location(id='url', refresh=False),
                html.Div([
                    dbc.Row([
                        nav_bar,
                        html.Hr(),
                    ]),    
                    dbc.Row([
                        html.Div([content]),
                    ],
                    align='start'),    
                ])         

    ],
    className="p-5",
    fluid=True,
)

chain.callback(app)
# chain_page_router(app)

@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return html.P("This is the content of the home page!")
    elif pathname == "/chain/black_metals":
        # return html.P("This is the content of page 1. Yay!")
        return chain.layout(pathname)
    elif pathname == "/basis" or pathname== "/cycle":
        return chain.layout(pathname)
    
    # If the user tries to reach a different page, return a 404 message
    # return html.Div(
    #     [
    #         html.H1("404: Not found", className="text-danger"),
    #         html.Hr(),
    #         html.P(f"The pathname {pathname} was not recognised..."),
    #     ],
    #     className="p-3 bg-light rounded-3",
    # )

# @app.callback(
#         Output("tab-symbol-rb", "children", allow_duplicate=True), 
#         [Input("url", "pathname")])
# def dispatch_page_content(pathname):
#     print('dispatch_page_content', pathname)    
#     if pathname=='/basis':
#         print(tab_variety)
#         return html.P("123")

if __name__ == "__main__":
    app.run_server(debug=True, port=9999)